In [2]:
from genetic_algorithm import *
import re

## hexadecimal generator

In [7]:
nr_of_input = 3
nr_of_actions = 3
nr_of_inner = 2
nr_of_genes = 8
nr_individuals = 4

res = generate_initial_genomes_for_population(nr_individuals, nr_of_genes, nr_of_input, nr_of_actions, nr_of_inner)

## Sumuj duble

In [10]:
dic = {}
for nr in res:
    dic[nr] = []
    for i in res[nr]:
        n_output = f'{i.output_id}{i.output_type}'
        n_input = f'{i.input_id}{i.input_type}'
        dic[nr].append([n_input, n_output, i.weight])

In [11]:
dic

{0: [['1mid', '0mid', 2.644895467724706],
  ['1mid', '1out', -1.0751716770944606],
  ['1mid', '2out', -3.0687623989012667],
  ['2in', '0mid', 1.405524187395086],
  ['1mid', '1mid', 3.8412940637875783],
  ['0in', '1mid', -1.766030825576072],
  ['1mid', '1mid', -2.6422096749580346],
  ['1in', '2out', 3.522050969021822]],
 1: [['0mid', '1out', 2.6663818098580805],
  ['0mid', '1mid', -2.7403631924309475],
  ['2in', '0out', 3.9472607965817184],
  ['2in', '2out', 0.2350068670837784],
  ['1in', '2out', 2.1958797497329465],
  ['0in', '1out', -3.1669159163741796],
  ['1in', '1out', 3.1674042423317563],
  ['1mid', '0out', 1.1672211200976652]],
 2: [['0in', '1out', 1.6280787425606593],
  ['1mid', '2out', -2.170364718449565],
  ['1mid', '1mid', 3.5744239279719214],
  ['0mid', '0out', 0.769967953609034],
  ['1mid', '1mid', -0.8037845261712193],
  ['0in', '0out', -2.8949183580039675],
  ['2in', '1mid', 2.8101938043644132],
  ['0in', '0out', 3.06241416145277]],
 3: [['0in', '0mid', -1.713047459179002

In [8]:
nr = 0

n_out = 0
pair_list = []
for pair in res[nr]:
    n_output = f'{pair.output_id}{pair.output_type}'
    n_input = f'{pair.input_id}{pair.input_type}'
    if pair.output_type == 'out' and pair.output_id > n_out:
        n_out = pair.output_id
    pair_list.append([n_input, n_output])


In [9]:
print(n_out)
pair_list

2


[['0in', '0mid'],
 ['2in', '1mid'],
 ['0mid', '1mid'],
 ['0in', '0out'],
 ['1mid', '1mid'],
 ['0mid', '0mid'],
 ['2in', '1mid'],
 ['1mid', '0mid'],
 ['0mid', '0out'],
 ['2in', '1out'],
 ['0mid', '2out'],
 ['1mid', '0mid'],
 ['0mid', '0mid'],
 ['0in', '0mid'],
 ['0in', '1out'],
 ['0in', '0mid']]

## grupuj 

In [4]:
dic

{0: {'0mid0out': -2.4527392034182816,
  '2in1out': 0.6819471997558371,
  '1mid1mid': 0.5615748512131848,
  '0mid1mid': 1.9885853807416451,
  '2in0out': 1.6363802838394628,
  '1mid0out': 3.5900503586143753,
  '1mid0mid': -0.5934381199450633,
  '0in0out': -0.060552418739508625,
  '1in1mid': 0.32839920647031895,
  '1in0out': 0.3309629177475965,
  '1mid2out': -0.7902334808484663},
 1: {'0mid0mid': -2.417945978940943,
  '2in1out': -3.955074011902945,
  '2in1mid': 3.791362734625362,
  '1mid2out': -0.9140241110941554,
  '1mid0out': 0.5950251793071876,
  '2in2out': 1.3392339386540515,
  '0mid1mid': -0.20753853197009003,
  '0in1out': -3.5414619258354954,
  '0mid1out': -2.6472150160231953,
  '0in0out': -1.0152296658019229,
  '1in1mid': -2.257530901877003,
  '1in2out': 2.3616664123302304},
 2: {'1mid2out': 1.927788799023348,
  '1in0mid': -1.1938348847855944,
  '1in2out': -0.7515336487105143,
  '1mid0out': -1.0256065924004272,
  '1in0out': 2.5771402411109414,
  '0in0out': 3.6756294826796885,
  '2i

In [5]:
# wybierz wszystkie in_mid

nr_neuronu = 3
print(dic[nr_neuronu])
list_of_keys = []
for i in dic[nr_neuronu]:
    if 'in' in i and 'mid' in i:
        print('in_mid', i)
        list_of_keys.append(i)
        
for i in list_of_keys:
    dic[nr_neuronu].pop(i)

print(dic[nr_neuronu])

{'1in2out': -3.0238364108042117, '2in1mid': -1.7905692049443003, '1mid2out': 1.9102090645505876, '0mid1out': -1.16209369754311, '0in0mid': -0.04053105447886464, '0mid0out': 2.7059362124217916, '0mid1mid': -0.8734930566152908, '1in0mid': 0.3086220051884633, '0mid2out': -1.1895620326567984, '0in2out': -3.9445750038150464, '2in2out': -2.4849687166183427, '0in1mid': -2.6258507553792154, '1mid1out': -0.8537158553334351, '1mid1mid': 2.4036624446818253}
in_mid 2in1mid
in_mid 0in0mid
in_mid 1in0mid
in_mid 0in1mid
{'1in2out': -3.0238364108042117, '1mid2out': 1.9102090645505876, '0mid1out': -1.16209369754311, '0mid0out': 2.7059362124217916, '0mid1mid': -0.8734930566152908, '0mid2out': -1.1895620326567984, '0in2out': -3.9445750038150464, '2in2out': -2.4849687166183427, '1mid1out': -0.8537158553334351, '1mid1mid': 2.4036624446818253}


In [6]:
dic[nr_neuronu]

{'1in2out': -3.0238364108042117,
 '1mid2out': 1.9102090645505876,
 '0mid1out': -1.16209369754311,
 '0mid0out': 2.7059362124217916,
 '0mid1mid': -0.8734930566152908,
 '0mid2out': -1.1895620326567984,
 '0in2out': -3.9445750038150464,
 '2in2out': -2.4849687166183427,
 '1mid1out': -0.8537158553334351,
 '1mid1mid': 2.4036624446818253}

In [7]:
list_of_out = ['0mid']

d = {}
for n in list_of_out:
    d[n] = [k for k in list_of_keys if n in k]
    
d

{'0mid': ['0in0mid', '1in0mid']}